In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from SimCLR_data_util import preprocess_for_train
from resnet_small import ResNet18
from tensorflow.keras.layers import Dense
from viewmaker import *
from SimCLR import *

In [2]:
tf.__version__

'2.6.0'

In [3]:
from datasets.cifar_10 import get_unsupervised_dataset
dataset = get_unsupervised_dataset(batch_size=64)

In [4]:
batch = next(iter(dataset))
batch_augmented = Viewmaker()(batch)
fix, (axis1, axis2) = plt.subplots(1, 2)
axis1.imshow(batch[0])
axis2.imshow(batch_augmented[0])

[Dimension(64), Dimension(32), Dimension(32), Dimension(3)] <class 'list'>
[Dimension(64), Dimension(8), Dimension(8), Dimension(128)] <class 'list'>
[Dimension(64), Dimension(8), Dimension(8), Dimension(129)] <class 'list'>
[Dimension(64), Dimension(8), Dimension(8), Dimension(130)] <class 'list'>


AttributeError: 'int' object has no attribute 'value'

In [5]:
def get_encoder():
    model = ResNet18(10)
    encoder = tf.keras.Sequential(model.layers[:-1])
    # encoder = tf.keras.Sequential([
    #     encoder, 
    #     tf.keras.layers.ReLU(), 
    #     tf.keras.layers.Dense(2048)
    # ])
    return encoder

encoder = get_encoder()

In [6]:
batch = next(iter(dataset))
output = encoder(batch)
encoder.summary()
print(f'input shape: {batch.shape}')
print(f'output shape: {output.shape}')

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (64, 32, 32, 64)          1728      
_________________________________________________________________
batch_normalization (BatchNo (64, 32, 32, 64)          256       
_________________________________________________________________
sequential_2 (Sequential)    (64, 32, 32, 64)          148480    
_________________________________________________________________
sequential_5 (Sequential)    (64, 16, 16, 128)         526848    
_________________________________________________________________
sequential_8 (Sequential)    (64, 8, 8, 256)           2102272   
_________________________________________________________________
sequential_11 (Sequential)   (64, 4, 4, 512)           8398848   
_________________________________________________________________
flatten (Flatten)            (64, 8192)              

In [14]:
batch = next(iter(dataset))
viewmaker = Viewmaker()
output = viewmaker(batch)
viewmaker.summary()
print(f'input shape: {batch.shape}')
print(f'output shape: {output.shape}')

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'next'

In [8]:
def get_projection_head():

    projection_head = tf.keras.Sequential([
        Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(256)
    ])

    return projection_head

projection_head = get_projection_head()

In [9]:
model = SimCLR_adversarial(encoder, viewmaker, projection_head)

NameError: name 'viewmaker' is not defined

In [10]:
BATCH_SIZE = 256 
EPOCHS = 10     
LEARNING_RATE = 0.0001
SAVE_DIRECTORY = './cifar_10_experiment'

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))

dataset = get_unsupervised_dataset(batch_size=BATCH_SIZE)  
dataset = dataset.take(len(dataset)-1)

model.fit(dataset, epochs=EPOCHS)

NameError: name 'model' is not defined

In [11]:
tf.shape(batch)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([64, 32, 32,  3], dtype=int32)>

In [12]:
Tensor()

NameError: name 'Tensor' is not defined

In [13]:
a = tf.constant(1.0, [64,32,32,3])
tf.shape(a)

TypeError: Cannot convert value [64, 32, 32, 3] to a TensorFlow DType.